<a href="https://colab.research.google.com/github/ebi19912/AI/blob/main/Fingerprint_CNN_Persian.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# نصب کتابخانه kaggle
!pip install kaggle

# آپلود فایل kaggle.json
from google.colab import files
files.upload()

# تنظیمات مربوط به فایل kaggle.json
!mkdir -p ~/.kaggle
!cp kaggle.json ~/.kaggle/
!chmod 600 ~/.kaggle/kaggle.json

# دانلود مجموعه داده از Kaggle
!kaggle datasets download -d peace1019/fingerprint-dataset-for-fvc2000-db4-b

# استخراج فایل‌ها از فایل فشرده
import zipfile
with zipfile.ZipFile('fingerprint-dataset-for-fvc2000-db4-b.zip', 'r') as zip_ref:
    zip_ref.extractall('dataset')


In [ ]:
import os
import cv2
import numpy as np
from tensorflow.keras.utils import to_categorical
from tensorflow.keras import models, layers

In [ ]:
# بارگیری و پردازش تصاویر
def load_images_from_folder(folder):
    images = []
    labels = []
    for filename in os.listdir(folder):
        img = cv2.imread(os.path.join(folder,filename), cv2.IMREAD_GRAYSCALE)
        if img is not None:
            images.append(img)
            labels.append(int(filename.split('_')[0].split('.')[0]))  # استخراج برچسب از نام فایل
    return images, labels

x_train, y_train = load_images_from_folder('/content/dataset/dataset_FVC2000_DB4_B/dataset/train_data')
x_test, y_test = load_images_from_folder('/content/dataset/dataset_FVC2000_DB4_B/dataset/real_data')

print("Number of training images:", len(x_train))
print("Shape of the first training image:", x_train[0].shape)


In [ ]:

# تبدیل لیست‌ها به آرایه‌های numpy و نرمال‌سازی
x_train = np.array(x_train).reshape(-1, 160, 160, 1).astype('float32') / 255.0
x_test = np.array(x_test).reshape(-1, 160, 160, 1).astype('float32') / 255.0


# تبدیل برچسب‌ها به فرمت one-hot
y_train = to_categorical(y_train)
y_test = to_categorical(y_test)


In [ ]:

# ایجاد شبکه عصبی
model = models.Sequential([
    layers.Conv2D(32, (3, 3), activation="relu", input_shape=(160, 160, 1)),
    layers.MaxPooling2D((2, 2)),
    layers.Conv2D(64, (3, 3), activation="relu"),
    layers.MaxPooling2D((2, 2)),
    layers.Flatten(),
    layers.Dense(128, activation="relu"),
    layers.Dense(10, activation="softmax"),
])


# آموزش شبکه عصبی
model.compile(optimizer="adam", loss="categorical_crossentropy", metrics=["accuracy"])
model.fit(x_train, y_train, epochs=10)

# ارزیابی شبکه عصبی
score = model.evaluate(x_test, y_test, verbose=0)
print("Test loss:", score[0])
print("Test accuracy:", score[1])

In [ ]:
!pip install joblib
import joblib

In [ ]:
# ثبت اثر انگشت
def register_fingerprint(image):
    # پیش پردازش تصویر
    image = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
    image = cv2.resize(image, (160, 160))
    image = image.reshape(-1, 160, 160, 1).astype('float32') / 255.0

    # پیش بینی ویژگی‌های اثر انگشت
    features = model.predict(image)

    # ذخیره ویژگی‌ها
    features = features.flatten()
    with open('fingerprint_features.pickle', 'wb') as f:
        joblib.dump(features, f)

# احراز هویت اثر انگشت
def authenticate_fingerprint(image):
    # پیش پردازش تصویر
    image = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
    image = cv2.resize(image, (160, 160))
    image = image.reshape(-1, 160, 160, 1).astype('float32') / 255.0

    # پیش بینی ویژگی‌های اثر انگشت
    features = model.predict(image)

    # مقایسه ویژگی‌ها
    with open('fingerprint_features.pickle', 'rb') as f:
        known_features = joblib.load(f)

    # استفاده از تابع فاصله برای مقایسه ویژگی‌ها
    dist = np.linalg.norm(features - known_features, axis=1)

    # نتیجه احراز هویت
    if dist < 0.01:
        return True
    else:
        return False


In [ ]:

# ثبت اثر انگشت
image = cv2.imread('FingerPrintOriginal.jpg')
register_fingerprint(image)

# احراز هویت اثر انگشت
image = cv2.imread('FingerPrintOriginal.jpg')
is_authenticated = authenticate_fingerprint(image)
print(is_authenticated)
